# River
runs through

In [ ]:
#| default_exp river

### Prior Art

In [ ]:
#| export
#standard
import numpy as np
import sys
import os
import math
import random

#data
from collections import namedtuple, deque
from dataclasses import dataclass,  field, asdict
from typing import List
from enum import Enum

#Jeremy
from dialoghelper import * 
from fastcore.basics import patch
from fasthtml.common import *
from fasthtml.jupyter import *
import httpx

# unique
from treelib import Tree
import heapq


In [ ]:
#| export


from HexMagic.styles import StyleCSS, SVGBuilder,SVGLayer, SVGPatternLoader, preview, app, StyleDemo
from HexMagic.primitives import MapCord, MapSize, MapRect, MapPath, Hex, HexGrid, HexWrapper, HexPosition, hexBackground,windy_edge, HexRegion
from HexMagic.terrain import  TerraDemo, Terrain
from HexMagic.terrainpatterns import TerrainPatterns


In [ ]:
#| export
from HexMagic.voronoi import PlateKind

#| export
### Helpers

In [ ]:
#| export
class RiverDemo:
    def __init__(self):
        self.help = ""

## River

The `River` class represents a river network as a tree structure using `treelib.Tree`.

**Core Data Structure**
- `tree`: A `treelib.Tree` where each node's `data` is a list of hex indices forming a continuous segment
- `hexes`: A flat `set` of all hex indices for quick intersection checks
- Root of tree = outlet (lowest point); branches go upstream toward sources

**Creating Rivers: `from_peak(terrain, peak_index)`**
1. Traces downhill from the peak using `terrain.lowest_neighbor()`
2. Stops when hitting water (elevation < 1) or a local minimum
3. Reverses the path so outlet is root
4. Creates a single-node tree with that path

**Merging Algorithm: `combine_rivers(rivers)`**
1. Loop until no changes occur
2. Check each pair of rivers for hex intersection
3. When found, call `_merge_with()` which:
   - Finds the intersection hex (picks highest elevation if multiple)
   - Locates which tree node contains it
   - Splits that node if needed (downstream stays, upstream becomes child)
   - Trims the other river's tree to just its upstream portion
   - Attaches it as a new branch at the intersection point

**Drawing: `drawRiver(river, riverStyle)`**
- Iterates through all tree nodes
- For each node, builds a list of center points from hex indices
- If the node has a parent, prepends the parent's first hex center (for continuity)
- Draws a spline through those points using `MapPath.drawSpline()`

The result: rivers that branch upstream from a single outlet, drawn as smooth curves through hex centers.

In [ ]:
#| export
class River:
    def __init__(self, terrain):
        self.terrain = terrain
        self.tree = Tree()
        self.hexes = set()  # All hex indices for quick intersection checks
        self.ocean_outlet = None

    @property
    def is_empty(self):
        return self.tree.size() == 0
    
    @classmethod
    def from_peak(cls, terrain, peak_index):
        """Create a river by tracing downhill from a peak."""
      
        
        # Trace downhill to find the outlet first
        path = [peak_index]
        current = peak_index
        
        while True:
            lowest = terrain.lowest_neighbor(current)
            # Stop if local minimum or hit water
            if lowest is None or terrain.elevations[lowest] < 1:
                break
            path.append(lowest)
            current = lowest
        # Don't create river if path is too short (just the peak itself)
        if len(path) < 2:
            return None
        
        # Build tree from outlet (root) upward
        # Reverse path so outlet is first
        path.reverse()
        
        river = River(terrain)
        # Create root node with the path as a single segment
        river.tree.create_node(tag="segment", identifier=0, data=path)
        river.hexes.update(path)
        
        return river
    
    @staticmethod
    def combine_rivers(rivers):
        """Merge intersecting rivers, return list with no intersections."""
        result = list(rivers)
        
        changed = True
        while changed:
            changed = False
            for i in range(len(result)):
                for j in range(i + 1, len(result)):
                    # Check for intersection
                    intersection = result[i].hexes & result[j].hexes
                    if intersection:
                        # Merge j into i
                        merged = result[i]._merge_with(result[j], intersection)
                        result[i] = merged
                        result.pop(j)
                        changed = True
                        break
                if changed:
                    break
        
        result = [r for r in result if not r.is_empty]
        return result
    
    def _merge_with(self, other, intersection):
        """Merge another river into this one at intersection point."""
        # TODO: implement tree merging logic
        # For now, just combine the hex sets
        merged = River(self.terrain)
        merged.hexes = self.hexes | other.hexes
        # Tree merging is more complex - need to attach other's
        # upstream portion at the intersection point
        return merged


In [ ]:
#| export
# In River class

@patch  
def show(self:River):
    """Display river tree structure with hex indices."""
    if self.tree.size() == 0:
        print("Empty river")
        return
    
    # Recursively print the tree with custom formatting
    def print_node(node, indent=0):
        # Print the hex indices for this segment
        indices_str = ", ".join(map(str, node.data))
        print("    " * indent + indices_str)
        
        # Print children
        children = self.tree.children(node.identifier)
        for child in children:
            print_node(child, indent + 1)
    
    # Start from root
    root = self.tree.get_node(self.tree.root)
    print_node(root)



In [ ]:
#| export
@patch
def midTerrain(self:RiverDemo):

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap =  Terrain(myBounds,radius=15,path = "volcano.svg")
    for center in [23,46,57,81,123,145,167,189,211,233,255]:
        sampleMap.volcano(center=center*3,adjusted=200,num_rings=8,variability=0.5,initial_threshold=0.4)
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    return sampleMap

@patch
def demoRiver(self:RiverDemo):
    sampleMap = self.midTerrain()
    peaks = sampleMap.find_peaks(40,1)
    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    rivers = River.combine_rivers(rivers)

    for river in rivers:
        print("====")
        river.show()
    



In [ ]:
RiverDemo().demoRiver()

### Merging Rivers

In [ ]:
#| export
@patch 
def _merge_with(self:River, other, intersection):
    """Merge another river into this one at intersection point."""
    merged = River(self.terrain)
    
    # Start with a copy of self's tree
    merged.tree = Tree(self.tree, deep=True)
    merged.hexes = self.hexes.copy()
    
    # Find the furthest upstream intersection point (highest elevation)
    intersection_hex = max(intersection, key=lambda h: self.terrain.elevations[h])
    
    for node in merged.tree.all_nodes():
        if intersection_hex in node.data:
            # Found the node containing intersection
            idx = node.data.index(intersection_hex)
            
            if idx == len(node.data) - 1:
                # Case 1: intersection at end of list
                attach_point = node.identifier
            else:
                # Case 2: intersection in middle - need to split
                # Keep [0:idx+1] in current node
                downstream = node.data[:idx+1]
                upstream = node.data[idx+1:]
                
                node.data = downstream
                
                # Create new child with upstream portion
                new_id = merged.tree.size()
                merged.tree.create_node(
                    tag="segment",
                    identifier=new_id,
                    parent=node.identifier,
                    data=upstream
                )
                
                # Move existing children to new node
                for child in merged.tree.children(node.identifier):
                    if child.identifier != new_id:
                        merged.tree.move_node(child.identifier, new_id)
                
                attach_point = node.identifier
            
            # Now trim and attach other tree
            trimmed_other = merged._trim_tree(other, intersection_hex)
            if trimmed_other:
                merged._attach_tree(trimmed_other, attach_point)
            
            break
    
    merged.hexes.update(other.hexes)
    return merged


@patch 
def _trim_tree(self:River, other, intersection_hex):
    """Return a copy of other's tree with everything at/below intersection removed."""
    # Find node containing intersection in other's tree
    for node in other.tree.all_nodes():
        if intersection_hex in node.data:
            idx = node.data.index(intersection_hex)
            
            # Keep only portion after intersection
            if idx == len(node.data) - 1:
                # Intersection at end, keep children
                # Return subtree starting from children
                trimmed = Tree()
                for child in other.tree.children(node.identifier):
                    trimmed = other.tree.subtree(child.identifier)
                    break  # For now just take first child, may need to handle multiple
                return trimmed
            else:
                # Split node, keep upstream portion
                trimmed = Tree(other.tree, deep=True)
                trim_node = trimmed.get_node(node.identifier)
                trim_node.data = trim_node.data[idx+1:]
                return trimmed
    
    return None
@patch 
def _attach_tree(self:River, other_tree, parent_id):
    """Attach other_tree's root(s) as children of parent_id."""
    if other_tree.size() == 0:
        return
    
    # Get next available ID
    next_id = self.tree.size()
    
    # Recursively copy nodes from other_tree
    def copy_subtree(other_node, new_parent_id):
        nonlocal next_id
        new_node_id = next_id
        next_id += 1
        
        self.tree.create_node(
            tag=other_node.tag,
            identifier=new_node_id,
            parent=new_parent_id,
            data=other_node.data.copy()
        )
        
        for child in other_tree.children(other_node.identifier):
            copy_subtree(child, new_node_id)
    
    # Copy from other's root
    other_root = other_tree.get_node(other_tree.root)
    copy_subtree(other_root, parent_id)


In [ ]:
#| export
@patch
def showIndices(self:Terrain):
    ret = ""
    i = 0
    for row in range(self.hexGrid.nRows):
        line = ""
        for col in range(self.hexGrid.nCols):
            line += f"{i:3} "
            i += 1
        ret += line + "\n"
    return ret


In [ ]:
#| export


@patch
def demoRiverMerge(self:RiverDemo):
    sampleMap = TerraDemo().tiny()
    print(sampleMap.showIndices())
    print()
    
    # Create three rivers manually
    river1 = River(sampleMap)
    river1.tree.create_node(tag="segment", identifier=0, data=[4, 5, 6])
    river1.hexes = {4, 5, 6}
    
    river2 = River(sampleMap)
    river2.tree.create_node(tag="segment", identifier=0, data=[5, 9, 14])
    river2.hexes = {5, 9, 14}
    
    river3 = River(sampleMap)
    river3.tree.create_node(tag="segment", identifier=0, data=[14, 11])
    river3.hexes = {14, 11}
    
    print("Before merge:")
    print("River 1:")
    river1.show()
    print("\nRiver 2:")
    river2.show()
    print("\nRiver 3:")
    river3.show()
    
    # Merge them
    rivers = [river1, river2, river3]
    merged = River.combine_rivers(rivers)
    
    print("\n\nAfter merge:")
    for i, river in enumerate(merged):
        print(f"\nMerged River {i+1}:")
        river.show()
    
    




In [ ]:
RiverDemo().demoRiverMerge()

  0   1   2   3 
  4   5   6   7 
  8   9  10  11 
 12  13  14  15 


Before merge:
River 1:
4, 5, 6

River 2:
5, 9, 14

River 3:
14, 11


After merge:

Merged River 1:
4, 5
    6
    9, 14
        11


### Working on Flow

Lets draw the rivers.

In [ ]:
#| export
@patch
def svg(self: River, styles=[StyleCSS("river", fill="none", stroke="blue", stroke_width=4)]) -> str:
    """Render river as SVG path.
    
    Args:
        styles: List of styles for different depth levels (thicker for main channels)
    
    Returns:
        SVG path string
    """
    ret = ""
    terrain = self.terrain
   
    for i, node in enumerate(self.tree.all_nodes()):
        points = []
        
        # If this node has a parent, add connection point
        parent_id = node.predecessor(self.tree.identifier)
        if parent_id is not None:
            parent_node = self.tree.get_node(parent_id)
            # Connect last point of child to first point of parent
            parent_first_hex = parent_node.data[-1]
            points.append(terrain.hexGrid.hexes[parent_first_hex].center)

        # Add all hex centers in this segment
        points.extend([terrain.hexGrid.hexes[idx].center for idx in node.data])
        
        # If this segment has ocean_outlet, extend to the border
        if self.ocean_outlet is not None:
            # Check if this is the outlet segment (last in path)
            if len(node.data) > 0:
                last_hex_idx = node.data[-1]
                last_hex = terrain.hexGrid.hexes[last_hex_idx]
                ocean_hex = terrain.hexGrid.hexes[self.ocean_outlet]
                
                # Add edge point (midpoint between last land hex and ocean hex)
                edge_point = MapCord(
                    (last_hex.center.x + ocean_hex.center.x) / 2,
                    (last_hex.center.y + ocean_hex.center.y) / 2
                )
                points.append(edge_point)

        # Select style based on tree depth (deeper = thicker)
        depth = self.tree.depth(node.identifier)
        style_index = min(depth, len(styles) - 1)
        
        # Create path and make it windy
        aPath = MapPath([], style=styles[style_index])
        aPath.points = points
        windy_river = aPath.make_windy(iterations=max(5 - i, 2), offset_factor=0.2)
        ret += windy_river.drawSpline()

    return ret


In [ ]:


@patch
def demoSanRivers(self:RiverDemo):
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)
    sampleMap = TerraDemo().sanFran()
    
    sampleMap.hexGrid.adjustRadius(20)
    aRender = sampleMap.hexGrid.builder
   
    peaks = sampleMap.find_peaks(35,7)

    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    colors = []
    for i, c  in enumerate(StyleCSS.seaborn("muted")):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c.properties["fill"] ,stroke_width=(i+1)*4)
        colors.append(aStyle)
        aRender.add_style(aStyle)
    
        
    #print(StyleCSS.generate(colors))
    # Merge them
    merged = River.combine_rivers(rivers)
    rivSVG = ""
    for i, v in enumerate(merged):
        rivSVG += v.svg(styles = colors)
        
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    sgrid = sampleMap.hexGrid
    sgrid.builder.adjust("regions", sgrid.styleLayer(f=windy_edge(iterations=2, offset_factor=0.1)))


    aRender.adjust("rivers", rivSVG)
    #aRender.adjust("root","")
    
    return aRender.show()
    


In [ ]:
RiverDemo().demoSanRivers()

## Erode

In [ ]:
#| export
@patch
def getLargestRiver(self:Terrain, num_peaks=40, min_height=1):
    """Find peaks, create rivers, merge them, and return the largest one."""
    peaks = self.find_peaks(num_peaks, min_height)
    rivers = [River.from_peak(self, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]
    merged = River.combine_rivers(rivers)
    
    if not merged:
        return None
    
    # Return river with most hexes
    return max(merged, key=lambda r: len(r.hexes))


1. I do think we can do a single pass (and always put it in a loop if we need more)
2. I think a wider valley would look better.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Perfect! Single pass with wider valleys it is.

**Here's my thinking for the algorithm:**

For each hex in the river tree, we'll:
1. Calculate its **flow accumulation** (count of upstream hexes feeding into it)
2. Calculate **gradient** (elevation drop to next downstream hex)
3. Create an erosion value: `erosion = base_factor * flow_accumulation * gradient`
4. Apply erosion to the hex AND its neighbors (with decay by distance)

For the last few hexes (near water/flat areas), switch to deposition instead.

**Structure suggestion:**
```python
@patch
def erode(self:River, base_erosion=0.5, valley_width=2, deposit_length=3):
    """Create elevation adjustments for river erosion."""
    adjustments = {}  # hex_index -> elevation_change (negative for erosion)
    
    # Your implementation here
    # - Walk the tree calculating flow accumulation
    # - Apply erosion based on gradient and flow
    # - Spread to neighbors within valley_width
    # - Handle deposition at the end
    
    return adjustments
```

Does this structure make sense? Should I walk you through calculating flow accumulation first, or would you like to try implementing part of it?

In [ ]:
#| export
@patch
def _calculate_flow(self:River):
    """Calculate accumulated flow for each hex in the river.
    Returns dict mapping hex_index -> flow_count"""
    flow = {}
    
    def traverse(node):
        # Start with hexes in this segment
        segment_flow = {}
        
        # Get flow from all children first (depth-first)
        child_flows = []
        for child in self.tree.children(node.identifier):
            child_flows.append(traverse(child))
        
        # Process this segment from end to beginning (upstream to downstream)
        for i in range(len(node.data) - 1, -1, -1):
            hex_idx = node.data[i]
            
            if i == len(node.data) - 1:
                # Last hex in segment - accumulate from children
                accumulated = 1  # This hex itself
                for child_flow in child_flows:
                    # Get flow from first hex of each child
                    accumulated += child_flow.get(node.data[i], 0)
            else:
                # Middle of segment - accumulate from next hex
                accumulated = 1 + segment_flow[node.data[i + 1]]
            
            segment_flow[hex_idx] = accumulated
            flow[hex_idx] = accumulated
        
        return segment_flow
    
    root = self.tree.get_node(self.tree.root)
    traverse(root)
    return flow


In [ ]:
def drawLegend(styles:[StyleCSS]):
    ret = ""
    
    radius = 15
    pad = 10
    x = radius + pad
    y = radius + pad
    
    for style in styles:
        # Use the actual style name for the class
        name = " ".join(style.name.split("_"))
        
        ret += f"""\t<circle cx="{x}" cy="{y}" r="{radius}" class="{style.name}"/>\n"""
        ret += f"""\t<text x="{x + pad + radius}" y="{y}" text-anchor="start" dy="0.3em" class="keyLabel">{name}</text>\n"""
        y += radius * 2 + pad

    return ret

        
        

In [ ]:
@patch
def demoFlow(self:RiverDemo):

    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)
    sampleMap = TerraDemo().sanFran()
    
    sampleMap.hexGrid.adjustRadius(15)
    aRender = sampleMap.hexGrid.builder
   
    peaks = sampleMap.find_peaks(35,7)

    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    colors = []
    legends = []
    for i, c  in enumerate(fills):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c ,stroke_width=4)
        colors.append(aStyle)
        aRender.add_style(aStyle)

        aStyle = StyleCSS(f"Level_{i+1}",fill=c,stroke="#000000" ,stroke_width=2)
        legends.append(aStyle)
        aRender.add_style(aStyle)
    
    #print(StyleCSS.generate(colors))
    # Merge them
    merged = River.combine_rivers(rivers)
    rivSVG = ""
    

    flowData = np.zeros(len(sampleMap.hexGrid.hexes)) - 1

    for i, stream in enumerate(merged):
        rivSVG += stream.svg(styles = colors)
        flows = stream._calculate_flow()
        streamFlow = np.zeros(len(sampleMap.hexGrid.hexes)) - 1
        for k,v in flows.items():
            streamFlow[k] = v
        flowData = np.maximum(flowData,streamFlow)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    
    flowData = [int(x) for x in flowData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(max(flowData),fills=fills)  # 5 levels
    sampleMap.makeOverlay(flowData, patterns)
    
    sampleMap.addCoast()
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    aRender.adjust("rivers", rivSVG)
    aRender.adjust("root","")
    aRender.adjust("legend",drawLegend(legends))
    

    return aRender.show()


In [ ]:

    

RiverDemo().demoFlow()

In [ ]:
#| export
@patch
def _calculate_gradient(self:River):
    """Calculate elevation gradient for each hex in the river.
    Returns dict mapping hex_index -> elevation_drop"""
    gradient = {}
    
    def traverse(node):
        # Process each hex in this segment
        for i in range(len(node.data)):
            hex_idx = node.data[i]
            current_elev = self.terrain.elevations[hex_idx]
            
            # Find next downstream hex
            if i > 0:
                # Next hex is earlier in the list (toward outlet)
                next_hex = node.data[i - 1]
            else:
                # At start of segment - check parent
                parent_id = node.predecessor(self.tree.identifier)
                if parent_id is not None:
                    parent_node = self.tree.get_node(parent_id)
                    next_hex = parent_node.data[-1]  # Last hex of parent
                else:
                    # At outlet - no gradient
                    gradient[hex_idx] = 0
                    continue
            
            next_elev = self.terrain.elevations[next_hex]
            drop = max(0, current_elev - next_elev)  # Only positive drops
            gradient[hex_idx] = drop
        
        # Recurse to children
        for child in self.tree.children(node.identifier):
            traverse(child)
    
    root = self.tree.get_node(self.tree.root)
    traverse(root)
    return gradient


In [ ]:
#| export
@patch
def demoGradient(self:RiverDemo):

    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)
    sampleMap = TerraDemo().sanFran()
    
    sampleMap.hexGrid.adjustRadius(15)
    aRender = sampleMap.hexGrid.builder
   
    peaks = sampleMap.find_peaks(35,7)

    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    colors = []
    legends = []
    for i, c  in enumerate(fills):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c ,stroke_width=4)
        colors.append(aStyle)
        aRender.add_style(aStyle)

        aStyle = StyleCSS(f"Level_{i+1}",fill=c,stroke="#000000" ,stroke_width=2)
        legends.append(aStyle)
        aRender.add_style(aStyle)
    
    #print(StyleCSS.generate(colors))
    # Merge them
    merged = River.combine_rivers(rivers)
    rivSVG = ""
    

    flowData = np.zeros(len(sampleMap.hexGrid.hexes)) - 1

    for i, stream in enumerate(merged):
        rivSVG += stream.svg(styles = colors)
        flows = stream._calculate_gradient()
        streamFlow = np.zeros(len(sampleMap.hexGrid.hexes)) - 1
        for k,v in flows.items():
            streamFlow[k] = v
        flowData = np.maximum(flowData,streamFlow)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    
    flowData = [int(x) for x in flowData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(max(flowData),fills=fills)  # 5 levels
    sampleMap.makeOverlay(flowData, patterns)
    
        
    sampleMap.colorMap()
    sampleMap.hexGrid.update()

    aRender.adjust("rivers", rivSVG)
    aRender.adjust("root","")
    sampleMap.addCoast()
    aRender.adjust("legend",drawLegend(legends))
    
    #return max(flowData)
    return aRender.show()



In [ ]:

RiverDemo().demoGradient()

In [ ]:

@patch
def demoPower(self:RiverDemo):

    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)
    sampleMap = TerraDemo().sanFran()
    
    sampleMap.hexGrid.adjustRadius(15)
    aRender = sampleMap.hexGrid.builder
   
    peaks = sampleMap.find_peaks(35,7)

    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    colors = []
    legends = []
    for i, c  in enumerate(fills):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c ,stroke_width=4)
        colors.append(aStyle)
        aRender.add_style(aStyle)

        aStyle = StyleCSS(f"Level_{i+1}",fill=c,stroke="#000000" ,stroke_width=2)
        legends.append(aStyle)
        aRender.add_style(aStyle)
    
    #print(StyleCSS.generate(colors))
    # Merge them
    merged = River.combine_rivers(rivers)
    rivSVG = ""
    
    powerData = np.zeros(len(sampleMap.hexGrid.hexes)) - 1

    for i, stream in enumerate(merged):
        rivSVG += stream.svg(styles = colors)
        flows = stream._calculate_flow()
        gradient = stream._calculate_gradient()

        streamPower = np.zeros(len(sampleMap.hexGrid.hexes)) - 1
        for k,v in flows.items():
            streamPower[k] = v * gradient[k]
        powerData = np.maximum(powerData,streamPower)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    
    powerData = [int(x) for x in powerData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(max(powerData),fills=fills)  # 5 levels
    sampleMap.makeOverlay(powerData, patterns)
    
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    sampleMap.addCoast()
    aRender.adjust("rivers", rivSVG)
    aRender.adjust("root","")
    aRender.adjust("legend",drawLegend(legends))
    
    #return max(flowData)
    return aRender.show()


In [ ]:
RiverDemo().demoPower()

now that I have the power how would I start to erode. There are two cases a v like river and a u shape glacier

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great question! The key difference is in how you spread the erosion to neighbors:

**V-shaped River (water erosion):**
- Erodes **most** at the river hex itself
- Falls off **quickly** to neighbors (exponential decay)
- Only affects hexes **lower** than the river hex (water cuts downward and slightly sideways)

**U-shaped Glacier:**
- Erodes **evenly** across the valley width
- Falls off **slowly** to neighbors (linear or gentle decay)
- Can affect hexes at **similar or higher** elevation (ice bulldozes everything)

Here's a structure to start:

```python
@patch
def erode(self:River, base_erosion=0.5, valley_width=2, shape='river'):
    """Apply erosion based on stream power.
    shape: 'river' for V-shaped, 'glacier' for U-shaped"""
    
    adjustments = {}
    flow = self._calculate_flow()
    gradient = self._calculate_gradient()
    
    for hex_idx in self.hexes:
        power = flow[hex_idx] * gradient[hex_idx]
        erosion_amount = base_erosion * power
        
        # Apply to river hex
        adjustments[hex_idx] = adjustments.get(hex_idx, 0) - erosion_amount
        
        # Spread to neighbors
        neighbors = self.terrain.hexGrid.neighbors(hex_idx)
        for dist in range(1, valley_width + 1):
            # Get neighbors at this distance
            ring = self.terrain.hexGrid.ring(hex_idx, dist)
            
            if shape == 'river':
                # V-shape: exponential decay, only lower neighbors
                decay = erosion_amount * (0.5 ** dist)
                for n in ring:
                    if self.terrain.elevations[n] < self.terrain.elevations[hex_idx]:
                        adjustments[n] = adjustments.get(n, 0) - decay
            else:
                # U-shape: linear decay, all neighbors
                decay = erosion_amount * (1 - dist / (valley_width + 1))
                for n in ring:
                    adjustments[n] = adjustments.get(n, 0) - decay
    
    return adjustments
```

Does this match your thinking? Should we test it on the San Francisco terrain?

In [ ]:



def rainfall_selector_np(values: np.ndarray) -> np.ndarray:
    # bin edges: <5 returns 0, 5-12 returns 1, 12-24 returns 2, etc.
    bins = [0.1, 5, 12, 24, 48]
    return np.digitize(values, bins) - 1 

test_data = np.array([-1, 2, 8, 15, 30, 60, 0, 12, 48])

indices = rainfall_selector_np(test_data)
indices



# Create a terrain
sampleMap = TerraDemo().tiny()

# Generate random rainfall data
rainfall_data = np.random.uniform(-4, 60, len(sampleMap.elevations))
sampleMap.fields['rainfall'] = rainfall_data

# Get pattern indices
pattern_indices = rainfall_selector_np(rainfall_data)

# Create patterns and overlay
patternGen = TerrainPatterns(sampleMap)
patterns = patternGen.ballScale(len(pattern_indices),fills=["#007fff","#d4ff00ff","#ee00ffff","#ff0099ff","#00ff1eff"])  # 5 levels
sampleMap.makeOverlay(pattern_indices, patterns)
#sampleMap.colorMap()
#sampleMap.hexGrid.update()

# View it
#print(sampleMap.hexGrid.builder.xml())
sampleMap.hexGrid.builder.show()

Yes I would love to visualize the  gradient

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I'd be happy to help you visualize the gradient! However, I notice that due to the dialog being truncated (44 messages reduced to 1), I don't have the context about what specific gradient you're referring to.

Could you please clarify:
- What function or model are you working with?
- What type of gradient are you looking to visualize (e.g., gradient of a loss function, gradient field, gradient descent steps, etc.)?
- Do you have any existing code or data that I should work with?

Once you provide these details, I can help you create an appropriate visualization!

## Generate

In [ ]:
#| export
@patch
def erode(self:River, base_erosion=0.5, valley_width=2, shape='river'):
    """Apply erosion based on stream power.
    shape: 'river' for V-shaped, 'glacier' for U-shaped"""
    
    adjustments = {}
    flow = self._calculate_flow()
    gradient = self._calculate_gradient()
    
    for hex_idx in self.hexes:
        power = flow[hex_idx] * gradient[hex_idx]
        erosion_amount = base_erosion * power
        
        # Apply to river hex
        adjustments[hex_idx] = adjustments.get(hex_idx, 0) - erosion_amount
        
        # Spread to neighbors
        neighbors = self.terrain.hexGrid.neighbors(hex_idx)
        for dist in range(1, valley_width + 1):
            # Get neighbors at this distance
            ring = self.terrain.hexGrid.ring(hex_idx, dist)
            
            if shape == 'river':
                # V-shape: exponential decay, only lower neighbors
                decay = erosion_amount * (0.5 ** dist)
                for n in ring:
                    if self.terrain.elevations[n] < self.terrain.elevations[hex_idx]:
                        adjustments[n] = adjustments.get(n, 0) - decay
            else:
                # U-shape: linear decay, all neighbors
                decay = erosion_amount * (1 - dist / (valley_width + 1))
                for n in ring:
                    adjustments[n] = adjustments.get(n, 0) - decay
    
    return adjustments


In [ ]:
#| export
@patch
def flow_directions(self: Terrain) -> np.ndarray:
    """Return array where each hex points to its downhill neighbor, or -1 if minimum."""
    n = len(self.elevations)
    directions = np.full(n, -1, dtype=int)
    
    for i in range(n):
        lowest = self.lowest_neighbor(i)
        if lowest is not None and self.elevations[lowest] < self.elevations[i]:
            directions[i] = lowest
    
    return directions

In [ ]:
#| export
@patch
def flow_diagram(self: Terrain) -> str:
    """Return array where each hex points to its downhill neighbor, or -1 if minimum."""
    n = len(self.elevations)
    overlay = ""
    style = StyleCSS("arrow", stroke="black",stroke_width=1)
    self.builder.add_style(style)

    directions = self.flow_directions()
    for i, x in enumerate(directions):
        if 0 <= x < n and self.elevations[i] > 0:


    
            path = MapPath([ self.hexGrid.hexes[i].center, self.hexGrid.hexes[x].center], style)
    
        
            overlay += path.with_arrowhead() + "\n"
    
    return overlay

In [ ]:
#| export


def find_local_minima(self: Terrain) -> list[int]:
    """Return indices of land hexes that are local minima (potential lakes)."""
    directions = self.flow_directions()
    return [i for i in range(len(self.elevations)) 
            if directions[i] == -1 and self.elevations[i] > 0]


@patch
def find_drainage_path(self: Terrain, start: int) -> list[int]:
    """Find lowest-cost path from start to ocean using Dijkstra.
    
    Cost = elevation we'd need to carve (downhill = 0, uphill = diff).
    """
    
    
    ocean = {i for i in range(len(self.elevations)) if self.elevations[i] <= 0}
    if start in ocean:
        return [start]
    
    # (cost, current_hex, path)
    pq = [(0, start, [start])]
    visited = set()
    
    while pq:
        cost, current, path = heapq.heappop(pq)
        
        if current in visited:
            continue
        visited.add(current)
        
        if current in ocean:
            return path
        
        for neighbor in self.hexGrid.neighborsOf(current):
            if neighbor < 0 or neighbor in visited:
                continue
            
            # Cost = how much we'd need to lower neighbor to flow from current
            elev_curr = self.elevations[current]
            elev_neighbor = self.elevations[neighbor]
            
            # To flow, neighbor must be < current
            carve_cost = max(0, elev_neighbor - elev_curr + 1)
            
            new_cost = cost + carve_cost
            heapq.heappush(pq, (new_cost, neighbor, path + [neighbor]))
    
    return []  # No path found




In [ ]:
#| export
@patch
def carve_to_ocean(self: Terrain, num_lakes: int = 5, max_iters: int = 10) ->  list[River]:
    """Carve drainage using lowest-cost paths to ocean."""
    paths = []
    
    for iteration in range(max_iters):
        minima = find_local_minima(self)
        if len(minima) <= num_lakes:
            print(f"Done at iter {iteration}: {len(minima)} lakes")
            break
        
        # Keep highest elevation minima as lakes
        minima.sort(key=lambda i: self.elevations[i], reverse=True)
        drain_these = minima[num_lakes:]
        
        for lake_idx in drain_these:
            path = self.find_drainage_path(lake_idx)
            river = River(terrain=self, hexes=path)
            paths.append(river)
           
            
            if len(path) < 2:
                continue
            
            # Carve monotonically decreasing along path
            for i in range(len(path) - 1):
                curr, next_hex = path[i], path[i + 1]
                
                if self.elevations[next_hex] >= self.elevations[curr]:
                    self.elevations[next_hex] = self.elevations[curr] - 1
        
        new_count = len(find_local_minima(self))
        
    
    return  paths

NameError: name 'patch' is not defined

So carve to ocean is one of our more powerful techinques along with voronoi digrams to make terrain realistic. are there things we can do do improve. for instance right now it just changes a single path as opposed to more of a u or v shape valley.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great observation! You already have the building blocks — your `erode()` method handles exactly this with V-shaped (river) and U-shaped (glacier) decay patterns.

The key insight: after `carve_to_ocean` creates the drainage *path*, you could apply erosion to *widen* it into a valley.

Two approaches to consider:

1. **Post-process**: After carving, create a temporary `River` from each path and call `erode()` on it
2. **Inline widening**: Modify `carve_to_ocean` to spread the carving to neighbors as it goes

Which feels more natural to you? The first keeps concerns separated (path-finding vs. valley-shaping), while the second might give more control over how depth relates to width.

---

*Note: Some earlier messages were removed due to context limits (originally 48, now 11). Let me know if you need me to catch up on anything.*

so i guess the easiest thing would be carve to ocean to return a list of rivers as opposed to a list of points. can you fix that

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Yes, that's the cleanest approach! Instead of returning `list[list[int]]`, return `list[River]`.

The change is minimal — at the end where you append each path, create a `River` object instead:

```python
river = River(terrain=self, hexes=path)
paths.append(river)
```

You'll also need to update the return type annotation to `list[River]`.

Want to try making that change, or would you like me to write the full updated method?

---

*Note: Due to context limits, some earlier messages were removed (originally 49, now 12). Let me know if I'm missing important context.*

In [ ]:
@patch
def demoPower(self:RiverDemo):

    fills=["#d4ff00eb","#ffb300ff","#ff0073ff","#9900ff97","#1e0e45eb"]
    riverStyle = StyleCSS("river",fill="none",stroke="blue",stroke_width=4)

    mySize = MapSize(480,480)
    myBounds = MapRect(MapCord(0,0), mySize)
    sampleMap, plates =  Terrain.fromSeeds(myBounds,radius=15)
    #sampleMap = TerraDemo().sanFran()
    sampleMap.carve_to_ocean(2)
    
    sampleMap.hexGrid.adjustRadius(15)
    aRender = sampleMap.hexGrid.builder
   
    peaks = sampleMap.find_peaks(35,7)

    rivers = [River.from_peak(sampleMap, peak) for peak in peaks]
    rivers = [r for r in rivers if r is not None]  # Filter out None rivers
    colors = []
    legends = []
    for i, c  in enumerate(fills):
        aStyle = StyleCSS(f"river{i}",fill="none",stroke=c ,stroke_width=4)
        colors.append(aStyle)
        aRender.add_style(aStyle)

        aStyle = StyleCSS(f"Level_{i+1}",fill=c,stroke="#000000" ,stroke_width=2)
        legends.append(aStyle)
        aRender.add_style(aStyle)
    
    #print(StyleCSS.generate(colors))
    # Merge them
    merged = River.combine_rivers(rivers)
    rivSVG = ""
    
    powerData = np.zeros(len(sampleMap.hexGrid.hexes)) - 1

    for i, stream in enumerate(merged):
        rivSVG += stream.svg(styles = colors)
        flows = stream._calculate_flow()
        gradient = stream._calculate_gradient()

        streamPower = np.zeros(len(sampleMap.hexGrid.hexes)) - 1
        for k,v in flows.items():
            streamPower[k] = v * gradient[k]
        powerData = np.maximum(powerData,streamPower)

    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    
    powerData = [int(x) for x in powerData]
    # Create patterns and overlay
    patternGen = TerrainPatterns(sampleMap)
    patterns = patternGen.ballDensity(max(powerData),fills=fills)  # 5 levels
    sampleMap.makeOverlay(powerData, patterns)
    
    sampleMap.colorMap()
    sampleMap.hexGrid.update()
    sampleMap.addCoast()
    aRender.adjust("rivers", rivSVG)
    aRender.adjust("root","")
    aRender.adjust("legend",drawLegend(legends))
    
    #return max(flowData)
    return aRender.show()

In [ ]:
RiverDemo().demoPower()

Done at iter 1: 1 lakes


# Soil

In [ ]:
#| export
@patch
def compute_distance_to_coast(self: Terrain):
    """Calculate distance from coast for each hex using BFS."""
   
    
    n_hexes = len(self.elevations)
    distances = np.full(n_hexes, np.inf)
    
    # Find all coastal hexes (at or below sea level)
    coastal = [i for i in range(n_hexes) if self.elevations[i] <= 0]
    
    # BFS to calculate distances
    queue = deque()
    for coast_idx in coastal:
        distances[coast_idx] = 0
        queue.append((coast_idx, 0))
    
    visited = set(coastal)
    
    while queue:
        current_idx, dist = queue.popleft()
        
        # Check all neighbors
        neighbors = self.ring(current_idx, 1)
        for neighbor in neighbors:
            if neighbor >= 0 and neighbor not in visited:
                distances[neighbor] = dist + 1
                visited.add(neighbor)
                queue.append((neighbor, dist + 1))
    
    self.fields['distance_to_coast'] = distances.astype(float)
    return self.fields['distance_to_coast']

In [ ]:
#| export
@dataclass
class SoilType:
    """Represents a soil/bedrock type with erosion properties."""
    name: str
    bedrock: str  # Description of rock type
    erosion_resistance: float  # 0-1, higher = harder to erode
    permeability: float  # 0-1, higher = water flows through faster
    color: str  # Hex color for visualization
    
    @classmethod
    def standard_types(cls) -> List['SoilType']:
        """Five standard soil types from hard rock to sediment."""
        return [
            cls(
                name="Granite",
                bedrock="Plutonic igneous (continental crust)",
                erosion_resistance=0.9,
                permeability=0.2,
                color="#8B7355"  # Gray-brown
            ),
            cls(
                name="Basalt",
                bedrock="Volcanic igneous (oceanic crust)",
                erosion_resistance=0.85,
                permeability=0.3,
                color="#4A4A4A"  # Dark gray
            ),
            cls(
                name="Limestone",
                bedrock="Sedimentary carbonate (dissolves)",
                erosion_resistance=0.5,
                permeability=0.6,
                color="#D4C5B9"  # Light tan
            ),
            cls(
                name="Sandstone",
                bedrock="Sedimentary clastic (crumbles)",
                erosion_resistance=0.4,
                permeability=0.7,
                color="#C2B280"  # Sandy tan
            ),
            cls(
                name="Alluvial",
                bedrock="Deposited sediment (clay/silt)",
                erosion_resistance=0.1,
                permeability=0.4,
                color="#8B6914"  # Dark gold/brown
            ),
        ]

In [ ]:
#| export
@patch
def to_style(self: SoilType, stroke_width: float = 1.0, opacity: float = 1.0) -> StyleCSS:
    """Create a StyleCSS for this soil type.
    
    Args:
        stroke_width: Width of stroke for hex borders
        opacity: Opacity of the fill color (0-1)
    
    Returns:
        StyleCSS with soil color as fill
    """
    return StyleCSS(
        name=f"soil_{self.name.lower()}",
        fill=self.color,
        stroke="#000000",
        stroke_width=stroke_width,
        opacity=opacity
    )


In [ ]:
#| export
@dataclass
class SoilSystem:
    """Manages soil types and their distribution across terrain."""
    terrain: 'Terrain'
    types: List[SoilType]
    regions: List[HexRegion]  # One per type (index matches types)
    
    def __post_init__(self):
        """Ensure terrain has soil_type field."""
        if 'soil_type' not in self.terrain.fields:
            self.terrain.fields['soil_type'] = np.zeros(len(self.terrain.elevations), dtype=int)

    @classmethod
    def from_plates(cls, terrain: 'Terrain', plates: List['Plate'], 
                    elev_threshold: float = 50.0,
                    precip_threshold: float = 750.0,
                    debug: bool = False) -> 'SoilSystem':
        """Create soil system from plate tectonics and terrain properties.
        
        Args:
            terrain: Terrain with elevations and climate data
            plates: List of tectonic plates
            elev_threshold: Elevation below which alluvial can form (meters)
            precip_threshold: Precipitation above which limestone forms (mm/year)
            debug: Print diagnostic info
        
        Returns:
            SoilSystem with initial soil distribution
        """

        
        types = SoilType.standard_types()
        n_hexes = len(terrain.elevations)
        
        # Initialize soil_type array
        soil_type = np.zeros(n_hexes, dtype=int)
        
        # Map plates to hexes
        plate_map = {}  # hex_idx -> plate
        for plate in plates:
            for hex_idx in plate.hexes:
                plate_map[hex_idx] = plate
        
        # Ensure we have distance_to_coast
        if 'distance_to_coast' not in terrain.fields:
            terrain.compute_distance_to_coast()
        
        # Track counts for debug
        counts = {i: 0 for i in range(len(types))}
        
        for i in range(n_hexes):
            elev = terrain.elevations[i]
            
            # Ocean hexes get basalt (oceanic crust)
            if elev <= 0:
                soil_type[i] = 1  # Basalt
                counts[1] += 1
                continue
            
            # Get plate type
            plate = plate_map.get(i)
            is_oceanic = plate and plate.kind == PlateKind.oceanic
            
            # Coastal lowlands → Alluvial
            if elev < elev_threshold and terrain.fields['distance_to_coast'][i] <= 1:
                soil_type[i] = 4  # Alluvial
                counts[4] += 1
            
            # High elevation → Bedrock (Granite or Basalt from plate)
            elif elev > 1000:
                if is_oceanic:
                    soil_type[i] = 1  # Basalt
                    counts[1] += 1
                else:
                    soil_type[i] = 0  # Granite
                    counts[0] += 1
            
            # Mid elevation → Sedimentary (Limestone or Sandstone)
            else:
                # Use precipitation if available
                if 'precipitation' in terrain.fields:
                    precip = terrain.fields['precipitation'][i]
                    if precip > precip_threshold:
                        soil_type[i] = 2  # Limestone (wet)
                        counts[2] += 1
                    else:
                        soil_type[i] = 3  # Sandstone (dry)
                        counts[3] += 1
                else:
                    # Fallback: use plate type
                    if is_oceanic:
                        soil_type[i] = 2  # Limestone
                        counts[2] += 1
                    else:
                        soil_type[i] = 3  # Sandstone
                        counts[3] += 1
        
        # Store in terrain
        terrain.fields['soil_type'] = soil_type
        
        # Create regions for each type
        regions = []
        for type_idx in range(len(types)):
            hexes = set(np.where(soil_type == type_idx)[0])
            regions.append(HexRegion(hexes=hexes, hexGrid=terrain.hexGrid))
        
        if debug:
            print("\n=== SOIL SYSTEM CREATED ===")
            print(f"Total hexes: {n_hexes}")
            for i, soil_type_obj in enumerate(types):
                pct = 100 * counts[i] / n_hexes
                print(f"{soil_type_obj.name:12s}: {counts[i]:5d} hexes ({pct:5.1f}%)")
        
        return cls(terrain=terrain, types=types, regions=regions)

In [ ]:
#| export
@patch
def soilOverlay(self:SoilSystem,f=None,smooth=False)->str:
    """ build an overlay simalar to HexGrid.styleLayer but uses plates."""
    retLayer = ""
    orders = {}  # Shared cache across all regions
    allPaths = []
    borders = {}  # Shared cache across all regions
    soilStyles = [x.to_style() for x in SoilType.standard_types()]
    

    for i, region in enumerate(self.regions):
        style = soilStyles[i]
        self.terrain.hexGrid.builder.add_style(style)
        allPaths.extend( region.trace_perimeter_cached(borders, f, style))

    allPaths = reversed(sorted(allPaths))
    for path in allPaths:
        if smooth:
            #path = path.smooth()
            retLayer += path.svg()
        else:
            retLayer += path.drawClosed()

    return retLayer

In [ ]:
#| export
def soilInformation(showText=True,terrain=None):
    global sampleTerrain, samplePlates
    if terrain == None:
        terrain = sampleTerrain
    terrain.colorMap()
    sGrid = terrain.hexGrid
    sGrid.builder.layers = []
    sampleSoil = SoilSystem.from_plates(terrain, samplePlates, debug=True)

    if showText:
        terrain.field_summary("soil_type")
        return
        

    sGrid.builder.adjust("regions", sampleSoil.soilOverlay(f=unique_windy_edge(iterations=2)))
  

    return sGrid.builder.show()

Lets think about carving vallies
from carve_to_ocean we have lists of drainages. and we have soils 
we need to compute 
1. fluvial erosion using our soils
2. knickpoint migration
3. fluvial deposition
4. updates to our soil

Can you build a simple model that uses what we have to adjust terrain elevations and our soil. we need an aging parameter for how low ie old world, new world.

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Here's a simple erosion model that ties together stream power, soil resistance, and age:

```python
@dataclass
class ErosionModel:
    """Simple erosion model with age parameter."""
    terrain: 'Terrain'
    soil_system: 'SoilSystem'
    age: float = 1.0  # 0.1 = young/new world, 1.0 = old world
    
    def erode(self, iterations: int = 1, base_rate: float = 5.0, deposition_threshold: float = 50.0):
        """Run erosion simulation.
        
        Args:
            iterations: Number of erosion passes (age scales this)
            base_rate: Base erosion amount per unit stream power
            deposition_threshold: Elevation below which deposition occurs
        """
        effective_iters = max(1, int(iterations * self.age))
        
        for _ in range(effective_iters):
            self._erode_pass(base_rate, deposition_threshold)
    
    def _erode_pass(self, base_rate: float, deposition_threshold: float):
        """Single erosion pass."""
        terrain = self.terrain
        soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
        types = self.soil_system.types
        
        # Get rivers and compute power
        peaks = terrain.find_peaks(40, 1)
        rivers = [River.from_peak(terrain, p) for p in peaks]
        rivers = [r for r in rivers if r]
        rivers = River.combine_rivers(rivers)
        
        adjustments = np.zeros(len(terrain.elevations))
        deposited = np.zeros(len(terrain.elevations))
        
        for river in rivers:
            flow = river._calculate_flow()
            gradient = river._calculate_gradient()
            
            for hex_idx in river.hexes:
                power = flow[hex_idx] * gradient[hex_idx]
                
                # Scale by soil resistance
                soil_idx = soil_types[hex_idx]
                resistance = types[soil_idx].erosion_resistance
                
                # Erosion = power * age / resistance
                erosion = base_rate * power * self.age / (resistance + 0.1)
                
                elev = terrain.elevations[hex_idx]
                
                if elev < deposition_threshold and gradient[hex_idx] < 5:
                    # Low gradient, low elevation → deposition
                    deposited[hex_idx] += erosion * 0.3
                else:
                    # Erosion
                    adjustments[hex_idx] -= erosion
                    
                    # Spread to valley walls (V-shape)
                    for neighbor in terrain.hexGrid.neighborsOf(hex_idx):
                        if neighbor >= 0 and terrain.elevations[neighbor] < elev:
                            adjustments[neighbor] -= erosion * 0.3 / (resistance + 0.1)
        
        # Apply changes
        terrain.elevations = np.clip(terrain.elevations + adjustments + deposited, 0, None)
        
        # Update soils based on new state
        self._update_soils(adjustments, deposited)
    
    def _update_soils(self, eroded: np.ndarray, deposited: np.ndarray):
        """Update soil types based on erosion/deposition."""
        soil_types = self.terrain.fields['soil_type']
        elevations = self.terrain.elevations
        
        for i in range(len(elevations)):
            if deposited[i] > 5:
                # Significant deposition → Alluvial
                soil_types[i] = 4
            elif eroded[i] < -20:
                # Heavy erosion → expose bedrock
                if elevations[i] > 500:
                    soil_types[i] = 0  # Granite
                else:
                    soil_types[i] = 1  # Basalt
            elif eroded[i] < -5:
                # Moderate erosion → sandstone/limestone
                soil_types[i] = 3 if soil_types[i] > 2 else 2
        
        # Rebuild regions
        for type_idx in range(len(self.soil_system.types)):
            hexes = set(np.where(soil_types == type_idx)[0])
            self.soil_system.regions[type_idx] = HexRegion(hexes=hexes, hex_grid=self.terrain.hexGrid)
```

**Usage:**

```python
@patch
def demoErosion(self: RiverDemo, age=0.5):
    """Demo erosion with age parameter. age: 0.1=young, 1.0=old"""
    mySize = MapSize(480, 480)
    myBounds = MapRect(MapCord(0, 0), mySize)
    terrain, plates = Terrain.fromSeeds(myBounds, slope=40, variation=90, radius=15)
    terrain.carve_to_ocean(2)
    
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    print(f"\n=== Before erosion (age={age}) ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    model = ErosionModel(terrain, soil, age=age)
    model.erode(iterations=3, base_rate=3.0)
    
    print(f"\n=== After erosion ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())
    
    return terrain.hexGrid.builder.show()
```

**Key behaviors by age:**
- **Young (0.1-0.3)**: Sharp peaks, minimal valleys, mostly original soils
- **Middle (0.4-0.6)**: V-shaped valleys forming, some exposed bedrock on slopes
- **Old (0.7-1.0)**: Wide valleys, alluvial deposits in lowlands, worn-down peaks

Want me to add knickpoint migration (waterfalls retreating upstream) as well?

please add knickpoint migration

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

```python
@patch
def _find_knickpoints(self: ErosionModel, river: River, gradient_threshold: float = 15.0) -> list[int]:
    """Find knickpoints (sudden gradient changes) in a river.
    
    Returns hex indices where gradient spikes (potential waterfalls).
    """
    gradient = river._calculate_gradient()
    knickpoints = []
    
    # Walk upstream looking for gradient jumps
    for node in river.tree.all_nodes():
        for i, hex_idx in enumerate(node.data):
            if gradient.get(hex_idx, 0) > gradient_threshold:
                knickpoints.append(hex_idx)
    
    return knickpoints

@patch
def _migrate_knickpoint(self: ErosionModel, hex_idx: int, river: River, migration_rate: float = 0.5):
    """Migrate a knickpoint upstream by eroding the lip.
    
    Args:
        hex_idx: Current knickpoint location
        river: The river containing this knickpoint
        migration_rate: How much to erode (scaled by age and soil)
    """
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get soil resistance at knickpoint
    resistance = types[soil_types[hex_idx]].erosion_resistance
    
    # Find upstream hex(es) - these form the "lip" of the waterfall
    upstream_hexes = []
    for node in river.tree.all_nodes():
        if hex_idx in node.data:
            idx = node.data.index(hex_idx)
            # Upstream is later in the list (remember: outlet is first)
            if idx < len(node.data) - 1:
                upstream_hexes.append(node.data[idx + 1])
            # Also check child nodes (tributaries joining here)
            for child in river.tree.children(node.identifier):
                upstream_hexes.append(child.data[0])
            break
    
    if not upstream_hexes:
        return
    
    # Calculate erosion at the lip
    current_elev = terrain.elevations[hex_idx]
    
    for upstream_idx in upstream_hexes:
        upstream_elev = terrain.elevations[upstream_idx]
        drop = upstream_elev - current_elev
        
        if drop > 5:  # Only erode if there's a real drop
            # Erosion inversely proportional to resistance
            erosion = migration_rate * self.age * drop / (resistance + 0.1)
            erosion = min(erosion, drop * 0.5)  # Don't over-erode
            
            # Erode the upstream lip
            terrain.elevations[upstream_idx] -= erosion
            
            # If we've eroded through soft rock to hard rock, slow down
            if erosion > 10 and soil_types[upstream_idx] in [3, 4]:  # Sandstone/Alluvial
                # Expose harder rock underneath
                soil_types[upstream_idx] = 2  # Limestone (harder)

@patch  
def _erode_pass(self: ErosionModel, base_rate: float, deposition_threshold: float):
    """Single erosion pass with knickpoint migration."""
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get rivers and compute power
    peaks = terrain.find_peaks(40, 1)
    rivers = [River.from_peak(terrain, p) for p in peaks]
    rivers = [r for r in rivers if r]
    rivers = River.combine_rivers(rivers)
    
    adjustments = np.zeros(len(terrain.elevations))
    deposited = np.zeros(len(terrain.elevations))
    
    for river in rivers:
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        # === KNICKPOINT MIGRATION ===
        knickpoints = self._find_knickpoints(river, gradient_threshold=12.0)
        for kp in knickpoints:
            self._migrate_knickpoint(kp, river, migration_rate=0.8)
        
        # === FLUVIAL EROSION ===
        for hex_idx in river.hexes:
            power = flow[hex_idx] * gradient[hex_idx]
            
            soil_idx = soil_types[hex_idx]
            resistance = types[soil_idx].erosion_resistance
            
            erosion = base_rate * power * self.age / (resistance + 0.1)
            
            elev = terrain.elevations[hex_idx]
            
            if elev < deposition_threshold and gradient[hex_idx] < 5:
                deposited[hex_idx] += erosion * 0.3
            else:
                adjustments[hex_idx] -= erosion
                
                for neighbor in terrain.hexGrid.neighborsOf(hex_idx):
                    if neighbor >= 0 and terrain.elevations[neighbor] < elev:
                        adjustments[neighbor] -= erosion * 0.3 / (resistance + 0.1)
    
    # Apply changes
    terrain.elevations = np.clip(terrain.elevations + adjustments + deposited, 0, None)
    
    self._update_soils(adjustments, deposited)
```

**What knickpoint migration does:**

1. **Find knickpoints**: Hexes where gradient exceeds threshold (steep drops = waterfalls)
2. **Erode the lip**: The upstream hex gets lowered, simulating waterfall retreat
3. **Soil interaction**: Soft rock erodes faster; when eroded through, exposes harder rock beneath (this is why waterfalls often form at rock layer boundaries)

**Age effect on knickpoints:**
- **Young terrain**: Knickpoints stay put, waterfalls are steep
- **Old terrain**: Knickpoints migrate far upstream, creating long gentle profiles

This creates the classic geological pattern where rivers start with stepped profiles and evolve toward smooth "graded" profiles over time.

In [ ]:
#| export
@dataclass
class ErosionModel:
    """Simple erosion model with age parameter."""
    terrain: 'Terrain'
    soil_system: 'SoilSystem'
    age: float = 1.0  # 0.1 = young/new world, 1.0 = old world
    
    def erode(self, iterations: int = 1, base_rate: float = 5.0, deposition_threshold: float = 50.0):
        """Run erosion simulation.
        
        Args:
            iterations: Number of erosion passes (age scales this)
            base_rate: Base erosion amount per unit stream power
            deposition_threshold: Elevation below which deposition occurs
        """
        effective_iters = max(1, int(iterations * self.age))
        
        for _ in range(effective_iters):
            self._erode_pass(base_rate, deposition_threshold)
    
    
    def _update_soils(self, eroded: np.ndarray, deposited: np.ndarray):
        """Update soil types based on erosion/deposition."""
        soil_types = self.terrain.fields['soil_type']
        elevations = self.terrain.elevations
        
        for i in range(len(elevations)):
            if deposited[i] > 5:
                # Significant deposition → Alluvial
                soil_types[i] = 4
            elif eroded[i] < -20:
                # Heavy erosion → expose bedrock
                if elevations[i] > 500:
                    soil_types[i] = 0  # Granite
                else:
                    soil_types[i] = 1  # Basalt
            elif eroded[i] < -5:
                # Moderate erosion → sandstone/limestone
                soil_types[i] = 3 if soil_types[i] > 2 else 2
        
        # Rebuild regions
        for type_idx in range(len(self.soil_system.types)):
            hexes = set(np.where(soil_types == type_idx)[0])
            self.soil_system.regions[type_idx] = HexRegion(hexes=hexes, hexGrid=self.terrain.hexGrid)


In [ ]:
#| export
@patch
def _find_knickpoints(self: ErosionModel, river: River, gradient_threshold: float = 15.0) -> list[int]:
    """Find knickpoints (sudden gradient changes) in a river.
    
    Returns hex indices where gradient spikes (potential waterfalls).
    """
    gradient = river._calculate_gradient()
    knickpoints = []
    
    # Walk upstream looking for gradient jumps
    for node in river.tree.all_nodes():
        for i, hex_idx in enumerate(node.data):
            if gradient.get(hex_idx, 0) > gradient_threshold:
                knickpoints.append(hex_idx)
    
    return knickpoints

@patch
def _migrate_knickpoint(self: ErosionModel, hex_idx: int, river: River, migration_rate: float = 0.5):
    """Migrate a knickpoint upstream by eroding the lip.
    
    Args:
        hex_idx: Current knickpoint location
        river: The river containing this knickpoint
        migration_rate: How much to erode (scaled by age and soil)
    """
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get soil resistance at knickpoint
    resistance = types[soil_types[hex_idx]].erosion_resistance
    
    # Find upstream hex(es) - these form the "lip" of the waterfall
    upstream_hexes = []
    for node in river.tree.all_nodes():
        if hex_idx in node.data:
            idx = node.data.index(hex_idx)
            # Upstream is later in the list (remember: outlet is first)
            if idx < len(node.data) - 1:
                upstream_hexes.append(node.data[idx + 1])
            # Also check child nodes (tributaries joining here)
            for child in river.tree.children(node.identifier):
                upstream_hexes.append(child.data[0])
            break
    
    if not upstream_hexes:
        return
    
    # Calculate erosion at the lip
    current_elev = terrain.elevations[hex_idx]
    
    for upstream_idx in upstream_hexes:
        upstream_elev = terrain.elevations[upstream_idx]
        drop = upstream_elev - current_elev
        
        if drop > 5:  # Only erode if there's a real drop
            # Erosion inversely proportional to resistance
            erosion = migration_rate * self.age * drop / (resistance + 0.1)
            erosion = min(erosion, drop * 0.5)  # Don't over-erode
            
            # Erode the upstream lip
            terrain.elevations[upstream_idx] -= erosion
            
            # If we've eroded through soft rock to hard rock, slow down
            if erosion > 10 and soil_types[upstream_idx] in [3, 4]:  # Sandstone/Alluvial
                # Expose harder rock underneath
                soil_types[upstream_idx] = 2  # Limestone (harder)

@patch  
def _erode_pass(self: ErosionModel, base_rate: float, deposition_threshold: float):
    """Single erosion pass with knickpoint migration."""
    terrain = self.terrain
    soil_types = terrain.fields.get('soil_type', np.zeros(len(terrain.elevations), dtype=int))
    types = self.soil_system.types
    
    # Get rivers and compute power
    peaks = terrain.find_peaks(40, 1)
    rivers = [River.from_peak(terrain, p) for p in peaks]
    rivers = [r for r in rivers if r]
    rivers = River.combine_rivers(rivers)
    
    adjustments = np.zeros(len(terrain.elevations))
    deposited = np.zeros(len(terrain.elevations))
    
    for river in rivers:
        flow = river._calculate_flow()
        gradient = river._calculate_gradient()
        
        # === KNICKPOINT MIGRATION ===
        knickpoints = self._find_knickpoints(river, gradient_threshold=12.0)
        for kp in knickpoints:
            self._migrate_knickpoint(kp, river, migration_rate=0.8)
        
        # === FLUVIAL EROSION ===
        for hex_idx in river.hexes:
            power = flow[hex_idx] * gradient[hex_idx]
            
            soil_idx = soil_types[hex_idx]
            resistance = types[soil_idx].erosion_resistance
            
            erosion = base_rate * power * self.age / (resistance + 0.1)
            
            elev = terrain.elevations[hex_idx]
            
            if elev < deposition_threshold and gradient[hex_idx] < 5:
                deposited[hex_idx] += erosion * 0.3
            else:
                adjustments[hex_idx] -= erosion
                
                for neighbor in terrain.hexGrid.neighborsOf(hex_idx):
                    if neighbor >= 0 and terrain.elevations[neighbor] < elev:
                        adjustments[neighbor] -= erosion * 0.3 / (resistance + 0.1)
    
    # Apply changes
    terrain.elevations = np.clip(terrain.elevations + adjustments + deposited, 0, None)
    
    self._update_soils(adjustments, deposited)


In [ ]:
@patch
def demoErosion(self: RiverDemo, age=0.5):
    """Demo erosion with age parameter. age: 0.1=young, 1.0=old"""
    mySize = MapSize(480, 480)
    myBounds = MapRect(MapCord(0, 0), mySize)
    terrain, plates = Terrain.fromSeeds(myBounds,  radius=15)
    terrain.carve_to_ocean(2)
    
    soil = SoilSystem.from_plates(terrain, plates, debug=True)
    
    print(f"\n=== Before erosion (age={age}) ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    model = ErosionModel(terrain, soil, age=age)
    model.erode(iterations=3, base_rate=3.0)
    
    print(f"\n=== After erosion ===")
    print(f"Max elev: {terrain.elevations.max():.0f}")
    
    # Visualize
    terrain.colorMap()
    terrain.hexGrid.update()
    terrain.addCoast()
    terrain.hexGrid.builder.adjust("soil", soil.soilOverlay())
    
    return terrain.hexGrid.builder.show()


In [ ]:
RiverDemo().demoErosion()

Done at iter 1: 2 lakes

=== SOIL SYSTEM CREATED ===
Total hexes: 1024
Granite     :     0 hexes (  0.0%)
Basalt      :   419 hexes ( 40.9%)
Limestone   :     0 hexes (  0.0%)
Sandstone   :   566 hexes ( 55.3%)
Alluvial    :    39 hexes (  3.8%)

=== Before erosion (age=0.5) ===
Max elev: 372

=== After erosion ===
Max elev: 369
